In [4]:
import pandas as pd
# import pandahouse as ph
import seaborn as sns


# import telegram
import io

import matplotlib.pyplot as plt
%matplotlib inline

connection = {'host': '',
                      'database':'',
                      'user':'', 
                      'password':''
                     }


# bot = telegram.Bot(token = '')

my_chat_id = 149956060
my_bot_id = 5058001314
reports_chat_id = -770113521
# alerts_chat_id = -1001706798154

In [9]:
pd.Timestamp('now').date()

datetime.date(2022, 8, 19)

In [13]:
pd.Timestamp(2017, 1,1).month_name()

'January'

In [16]:
pd.Timestamp(2017, 1,1) - pd.Timedelta(days=1)

Timestamp('2016-12-31 00:00:00')

In [17]:
pd.Timestamp(2017, 1,1) - pd.DateOffset(days=1)

Timestamp('2016-12-31 00:00:00')

In [15]:
pd.Timestamp(2017, 1,1).to_pydatetime()

datetime.datetime(2017, 1, 1, 0, 0)

In [25]:
# Получить chat-id разными способами:

updates = bot.getUpdates()
print(updates[-1])




{'update_id': 716842662, 'message': {'message_id': 7, 'new_chat_members': [], 'chat': {'type': 'private', 'id': 149956060, 'last_name': 'Kandybko', 'first_name': 'Daria', 'username': 'kandydaria'}, 'entities': [], 'date': 1660231272, 'group_chat_created': False, 'photo': [], 'caption_entities': [], 'text': 'fdrf', 'new_chat_photo': [], 'supergroup_chat_created': False, 'channel_chat_created': False, 'delete_chat_photo': False, 'from': {'first_name': 'Daria', 'language_code': 'ru', 'is_bot': False, 'id': 149956060, 'last_name': 'Kandybko', 'username': 'kandydaria'}}}


In [26]:
query = '''
select toDate(time) as day, user_id, action
from simulator_20220720.feed_actions
where toDate(time) between  today() - 7 and today()-1;
'''

df = ph.read_clickhouse(query, connection=connection)

In [27]:
df

,day,user_id,action
0,2022-08-10,126581,view
1,2022-08-10,141796,view
2,2022-08-10,147578,view
3,2022-08-10,156973,like
4,2022-08-10,164863,view
...,...,...,...
5678766,2022-08-10,108633,view
5678767,2022-08-10,115307,view
5678768,2022-08-10,117768,view
5678769,2022-08-10,123018,like


In [28]:
data = pd.merge(df.groupby('day')['user_id'].nunique().reset_index(),
         df.groupby(['day','action']).count().unstack().droplevel(level=0, axis=1).reset_index(),
         how = 'inner', 
         on = 'day')

data.columns = ['day','DAU','likes','views']
data['CTR'] = data['likes'] / data['views']
data

,day,DAU,likes,views,CTR
0,2022-08-04,19026,100781,513026,0.196444
1,2022-08-05,18760,131364,626809,0.209576
2,2022-08-06,18885,128146,604637,0.211939
3,2022-08-07,18708,156523,737226,0.212313
4,2022-08-08,18494,140421,687869,0.204139
5,2022-08-09,18984,164457,767819,0.214187
6,2022-08-10,18902,160496,759197,0.211402


In [29]:
report = f'''NEWSFEED \nreport for {data.iloc[-1,:].day.strftime('%d-%m-%Y')}: \n 
DAU: {data.iloc[-1,1]} 
Views: {data.iloc[-1,3]}  
Likes: {data.iloc[-1,2]}  
CTR: {data.iloc[-1,4].round(4)}  \n
'''

In [30]:
report

'NEWSFEED \nreport for 10-08-2022: \n \nDAU: 18902 \nViews: 759197  \nLikes: 160496  \nCTR: 0.2114  \n\n'

In [34]:
sns.set(font_scale=1,
       style="whitegrid",
       rc={'figure.figsize':(10,13)})

fig, axes = plt.subplots(nrows=4, ncols=1, sharex=True)


sns.lineplot(x='day', y='DAU', data = data, ax = axes[0], color = 'lightpink', marker="^")
sns.lineplot(x='day', y='likes', data = data, ax = axes[1], color = 'purple', marker="^")
sns.lineplot(x='day', y='views', data = data, ax = axes[2], color = 'orchid', marker="^")
sns.lineplot(x='day', y='CTR', data = data, ax = axes[3], color = 'tomato', marker="^")


for name, ax in zip([i for i in data][1:], axes.flatten()):
    ax.set_frame_on(False)
    ax.grid(True, color='#e2e2e2', alpha=0.6)
    ax.set(title = name, ylabel='')
    
ax.set_xlabel('\n')

ax.set_xticklabels(list(data['day'].apply(lambda x: x.strftime('%d %b'))))

fig.suptitle('\n\nNEWSFEED METRICS', fontsize=14)

plot_object = io.BytesIO()
plt.savefig(plot_object)
plot_object.name = 'newsfeed_metrics.png'
plot_object.seek(0)

plt.close()

/tmp/ipykernel_110/3189779541.py:21: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(list(data['day'].apply(lambda x: x.strftime('%d %b'))))


In [32]:
# bot.sendMessage(chat_id = my_chat_id, text=report)
# bot.sendPhoto(chat_id = my_chat_id, photo = plot_object)

In [35]:
bot.sendMessage(chat_id = reports_chat_id, text=report)
bot.sendPhoto(chat_id = reports_chat_id, photo = plot_object)